In [ ]:
import hpmcm
import tables_io
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATADIR = "shear_tract"
shear_st = "0p01"

tract = 10463
SOURCE_TABLEFILES = sorted(glob.glob(os.path.join(DATADIR, f"shear_*_{shear_st}_cleaned_{tract}_ns.pq")))
SOURCE_TABLEFILES.append(os.path.join(DATADIR, f"object_{tract}.pq"))
SOURCE_TABLEFILES.reverse()
SOURCE_TABLEFILES = [SOURCE_TABLEFILES[0], SOURCE_TABLEFILES[2]]
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

#REF_DIR = (37.91729, 6.94389)  # RA, DEC in deg
#REGION_SIZE = (1.5, 1.5)
REF_DIR = (37.9, 7.0)  # RA, DEC in deg
REGION_SIZE = (0.375, 0.375)


PIXEL_SIZE = 0.5/3600.
PIXEL_R2CUT = 4.
PIXEL_MATCH_SCALE = 1

In [ ]:
SOURCE_TABLEFILES

In [ ]:
matcher = hpmcm.WcsMatch.create(REF_DIR, REGION_SIZE, pixelSize=PIXEL_SIZE, pixelR2Cut=PIXEL_R2CUT)

In [ ]:
matcher.reduceData(SOURCE_TABLEFILES, VISIT_IDS)

In [ ]:
_ = plt.hist(matcher.fullData[0].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="obj")
_ = plt.hist(matcher.fullData[1].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="pgauss")
#_ = plt.hist(matcher.fullData[2].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="pgauss")
#_ = plt.hist(matcher.fullData[3].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="gauss")
_ = plt.xscale('log')
_ = plt.legend()

In [ ]:
matcher.nCell

In [ ]:
matcher.analysisLoop()

In [ ]:
idx = matcher.getCellIdx(2,2)
cell = matcher.cellDict[idx]
od = cell.analyze(None, 4)
_ = plt.imshow(od['countsMap'], origin='lower', cmap='gray_r')

In [ ]:
cluster = list(cell.clusterDict.values())[0]
_ = hpmcm.viz_utils.showCluster(od['image'], cluster, cell)

In [ ]:
objLists = hpmcm.classify.classifyObjects(matcher, snrCut=10)
hpmcm.classify.printObjectTypes(objLists)

In [ ]:
n_good = len(objLists['ideal'])
bad_list = ['edge_mixed', 'edge_missing', 'edge_extra', 'orphan', 'missing', 'two_missing', 'many_missing', 'extra', 'caught']
n_bad = np.sum([len(objLists[x]) for x in bad_list])

In [ ]:
effic = n_good/(n_good+n_bad)
effic_err = np.sqrt(effic*(1-effic)/(n_good+n_bad))
print(f"Effic: {effic:.5} +- {effic_err:.5f}")

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['missing'][0])

In [ ]:
_ = plt.scatter(matcher.fullData[1].pgauss_T, matcher.fullData[1].pgauss_psf_T)

In [ ]:
odict = hpmcm.classify.matchObjectsAgainstRef(matcher, snrCut=10.)

In [ ]:
hpmcm.classify.printObjectMatchTypes(odict)

In [ ]:
_ = plt.scatter(matcher.redData[0].ra, matcher.redData[0].dec, s=1)
_ = plt.scatter(matcher.redData[1].ra, matcher.redData[1].dec, s=1)